# sscan as Bluesky plan

## 1D step scans using sscan record

Support the [sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html) with a  [bluesky](http://nsls-ii.github.io/bluesky) plan for data acquisition.  Consider the case of [1D step scans using sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html#HEADING_1-1).

In [1]:
import asyncio
from collections import deque, OrderedDict
import numpy as np
import time

%matplotlib notebook
from bluesky.utils import install_qt_kicker
install_qt_kicker()

# common IOC prefix to be used
P = "prj:"

In [2]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH(f"{P}scaler1", name="scaler")
scaler.select_channels(None)

In [3]:
from ophyd import EpicsMotor
m1 = EpicsMotor(f"{P}m1", name="m1")

In [4]:
from apstools.synApps_ophyd import userCalcsDevice
calcs = userCalcsDevice(P, name="calcs")

In [5]:
from apstools.synApps_ophyd import sscanDevice
scans = sscanDevice(P, name="scans")
scans.select_channels()

In [6]:
from apstools.synApps_ophyd import SaveData
save_data = SaveData(f"{P}saveData_", name="save_data")

In [7]:
# configure saveData for data collection into MDA files:
        
save_data.file_system.put("/tmp")
save_data.subdirectory.put("saveData")
save_data.base_name.put("sscan1_")
save_data.next_scan_number.put(1)
save_data.comment1.put("testing")
save_data.comment2.put("configured and run from ophyd")

In [8]:
# configure the sscan record for data collection:

# clear out the weeds
scans.reset()

scan = scans.scan1
scan.number_points.put(6)
scan.positioners.p1.setpoint_pv.put(m1.user_setpoint.pvname)
scan.positioners.p1.readback_pv.put(m1.user_readback.pvname)
scan.positioners.p1.start.put(-1)
scan.positioners.p1.end.put(0)
scan.positioner_delay.put(0.0)
scan.detector_delay.put(0.1)
scan.detectors.d01.input_pv.put(scaler.channels.chan03.s.pvname)
scan.detectors.d02.input_pv.put(scaler.channels.chan02.s.pvname)
scan.triggers.t1.trigger_pv.put(scaler.count.pvname)

# finally, reconfigure
scans.select_channels()

In [9]:
# make a noisy detector in an EPICS swait record, peak ceneter at 2
from apstools.synApps_ophyd import swait_setup_lorentzian
swait_setup_lorentzian(calcs.calc2, m1, 2)
noisy_det = calcs.calc2.val
noisy_det.kind = "hinted"

In [10]:
def ophyd_step_scan(motor):
    """step-scan the motor and read the noisy detector"""
    t0 = time.time()
    for p in range(10):
        motor.move(p-3)
        print(
            "%8.3f" % (time.time()-t0), 
            "%8.2f" % motor.position, 
            "%8.4f" % noisy_det.get()
             )
    motor.move(0)
    print("Complete in %.3f seconds" % (time.time()-t0))

# ophyd_step_scan(m1)

--------
## setup Bluesky, databroker, and the RunEngine

In [11]:
from databroker import Broker
db = Broker.named("mongodb_config")

In [12]:
from bluesky import RunEngine
import bluesky.plans as bp
import bluesky.plan_stubs as bps
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky import SupplementalData

RE = RunEngine({})
RE.subscribe(db.insert)
RE.subscribe(BestEffortCallback())
RE.preprocessors.append(SupplementalData())

------
## Develop the BS plan

In [23]:
import apstools.plans as APS_plans
import ophyd


def get_sscan_data_objects(sscan):
    """
    prepare a dictionary of the "interesting" ophyd data objects for this scan
    """
    scan_data_objects = OrderedDict()
    for part in (sscan.positioners, sscan.detectors):
        for chname in part.read_attrs:
            if not chname.endswith("_value"):
                continue
            obj = getattr(part, chname)
            key = obj.name.lstrip(sscan.name + "_")
            scan_data_objects[key] = obj
    return scan_data_objects

    
def sscan_step_1D(sscan, _md={}):
    """simple 1-D step scan using EPICS synApps sscan record"""
    # TODO: prep
    t0 = time.time()
    sscan_status = ophyd.DeviceStatus(sscan.execute_scan)
    started = False
    
    def execute_cb(value, timestamp, **kwargs):
        if started and value in (0, "IDLE"):
            sscan_status._finished()
            sscan.execute_scan.unsubscribe_all()
            sscan.scan_phase.unsubscribe_all()
        else:
            elapsed = "%.3f" % (time.time() - t0)
            phase = sscan.scan_phase.get(as_string=True)
            print(f"{elapsed} execute_cb() {timestamp}: phase={phase}  executing={value}")
    
    def phase_cb(value, timestamp, **kwargs):
        elapsed = "%.3f" % (time.time() - t0)
        phase = sscan.scan_phase.enum_strs[value]
        #print(f"{elapsed} phase_cb() {timestamp}: phase={phase}  value={value}")
        if value in (15, "RECORD SCALAR DATA"):
            #yield sscan.read()
            #print(sscan.read())
            print(f"{elapsed} new data {timestamp}")
            # TODO: how to report this data FROM HERE to RE
    
    #yield from bps.open_run(_md)
    sscan.execute_scan.subscribe(execute_cb)
    sscan.scan_phase.subscribe(phase_cb)
    yield from bps.mv(sscan.execute_scan, 1)
    #report()
    started = True
    #yield from bps.close_run()

    
    # TODO: collect and emit data, wait for sscan to end
    return sscan_status

In [24]:
scans = sscanDevice(P, name="scans")
scans.select_channels()

RE(sscan_step_1D(scans.scan1))

0.001 execute_cb() 1552342298.804289: phase=IDLE  executing=0
0.004 execute_cb() 1552342298.804289: phase=SCAN_PENDING  executing=1
0.004 execute_cb() 1552342405.5305305: phase=SCAN_PENDING  executing=1


()

1.823 new data 1552342405.603196
2.721 new data 1552342405.603196
3.625 new data 1552342405.603196
4.528 new data 1552342405.603196
5.430 new data 1552342405.603196
6.339 new data 1552342405.603196
